In [1]:
%pip install -r requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
#import shap

In [86]:
excel_calidad = pd.read_excel(
    'CC_FT_17_Formato_de_Control_de_Calidad_Café_de_Trillado.xlsx',
    sheet_name = ['CONTROL CALIDAD CAFE TRILLADO J' , 'Sheet2'],
    skiprows=5,
    skipfooter=4
)

excel_calidad['CONTROL CALIDAD CAFE TRILLADO J'].columns = excel_calidad['CONTROL CALIDAD CAFE TRILLADO J'].columns.str.strip()
excel_calidad['Sheet2'].columns = excel_calidad['Sheet2'].columns.str.strip()
df_calidad = pd.concat(excel_calidad.values(), ignore_index=True)

df_calidad = df_calidad[[
    'LOTE', 'DENOMINACIÓN/     MARCA', 'CANTIDAD', '%H', 'MALLAS',
    'NOTAS DE CATACIÓN', 'PUNTAJE'
]]
df_calidad.dropna(subset=['LOTE'], inplace=True)

df_calidad

,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,NOTAS DE CATACIÓN,PUNTAJE
2,01-190722,Madre Laura,765.00,10.9,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
3,09-190722,Tabi Natural,204.00,10.2,14,"Frutas maduras, nibs de cacao, acidez brillant...",85
4,10-190722,Don Mario,165.00,10.7,14,"Panela, durazno, miel, acidez brillante citric...",84.5
5,07-19-07-22,Don Felix,0.45,10.5,14,"Moras maduras, chocolate negro, acidez media c...",84.5
6,01-291022,Madre Laura,105.00,10.7,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
...,...,...,...,...,...,...,...
120,01-271023,Madre Laura,20.00,10.5,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
121,01-100124,Madre Laura,20.00,10.4,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
122,01-020424,Madre Laura,20.00,10.5,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
123,01-200624,Madre Laura,14.00,10.8,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84


In [87]:
typos = {
    'fnal': 'final',
    'caramleo': 'caramelo',
    'cuepo': 'cuerpo',
    'balanaceado': 'balanceado'
}
typos_patron = r'\b(' + '|'.join(typos.keys()) + r')\b'

agr = {
    r'\b(arandanos|moras? maduras?|frambuesa|frutos rojos|fresas?|ciruela madura)\b':'bayas',
    r'\b(limon(?:aria|ciollo)?|mandarina|naranja|sandia|uva|durazno|manzana roja|frutas (?:amarillas|maduras))\b':'frutales',
    r'\bacidez\b.*':'acidez',
    r'\b(cuerpo\b.*|aterciopelado|cremoso(?: medio)?|sedoso)\b':'cuerpo',
    r'\b(cacao|nibs de cacao|chocolate(?: al \d+%|.*)?)\b':'chocolate',
    r'\b(dulce(?: de leche| prolongado| como la melaza)?|azucar morena|panela|melao de panela|miel(?: de maple)?)\b':'dulce',
    r'\b(jazmin|lavanda|rosas?|flor de naranja|final floral dulce)\b':'floral',
    r'\b(final(?: con notas a [\w\s]+)?|prolongado(?: a [\w\s]+)?)\b':'final',
    r'\b(clavos de olor|vainilla|nuez moscada|residual a cascara de mandarina)\b':'especias',
    r'\b(almendras|nueces?)\b':'nueces',
    r'\bresidual\b.*':'residual',
    r'\bjalea\b':'jalea',
    r'\b(cedro|te de cedron|menta)\b':'otros'
}

serie = (df_calidad.pop('NOTAS DE CATACIÓN')      # lo extraemos y quitamos del df
           .fillna('')                           
           .astype(str).str.lower()              
           .str.replace(typos_patron, lambda m: typos[m.group(0)], regex=True)
)
for pat, cat in agr.items():
    serie = serie.str.replace(pat, f',{cat},', regex=True)

# 2) limpieza de delimitadores en dos pasos
serie = (serie
    .str.replace(r'\s*[y,\.]+\s*', ',', regex=True)  # todo " y ", comas con espacios o puntos → ","
    .str.replace(r',+', ',',      regex=True)        # colapsa comas múltiples
    .str.strip(',')                                  # quita comas al inicio/fin
    .str.replace(r'\b(?:a)?\b', '', regex=True)      # elimina token "a" solitario
    .str.replace(r',+', ',',      regex=True)        
    .str.strip(',')
)

# 3) dummies + fusionar duplicados con un solo groupby
dummies = (
    serie.str.get_dummies(sep=',')
         .T.groupby(level=0).max() 
         .T
)

# 4) montamos el df final
df_calidad = pd.concat([df_calidad, dummies], axis=1)

df_calidad

,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,PUNTAJE,acidez,as,ba,caramelo,...,dulce,especias,flor de,floral,frutales,jalea,maracu,mora,nueces,otros
2,01-190722,Madre Laura,765.00,10.9,14,84,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,09-190722,Tabi Natural,204.00,10.2,14,85,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,10-190722,Don Mario,165.00,10.7,14,84.5,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
5,07-19-07-22,Don Felix,0.45,10.5,14,84.5,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
6,01-291022,Madre Laura,105.00,10.7,14,84,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,01-271023,Madre Laura,20.00,10.5,14,84,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
121,01-100124,Madre Laura,20.00,10.4,14,84,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
122,01-020424,Madre Laura,20.00,10.5,14,84,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
123,01-200624,Madre Laura,14.00,10.8,14,84,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
excel_tostion = pd.read_excel(
    'CC_FT_18_Formato_de_Tostión.xlsx',
    sheet_name=['TOSTIÓN JERICÓ L', 'TOSTIÓN JERICÓ'],
    skiprows=5,
    skipfooter=0
)

df_tostion = pd.concat(excel_tostion.values(), ignore_index=True)
df_tostion.columns = df_tostion.columns.str.strip()
df_tostion = df_tostion.drop(columns=['Fecha', 'Observaciones','Tostador'])

df_tostion

,Lote,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Temp. De inicio y final,Tiempo de tueste
0,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,15.000000,7.65,Filtrado,175°/191°,08:01:00
1,01-190722,Jerico,Dos mil,Tradicional,Lavado,3.00,16.666667,2.50,Espressso,175°/195°,08:42:00
2,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,16.444444,7.52,Filtrado,175°/190°,07:58:00
3,01-190722,Jerico,Dos mil,Tradicional,Lavado,11.70,16.324786,9.79,Filtrado,175°/191°,08:02:00
4,09-190722,Ciudad Bolivar,Tabi,Natural,Natural,0.45,24.444444,0.34,Filtrado,150°/186°,08:10:00
...,...,...,...,...,...,...,...,...,...,...,...
524,01-271023,Jerico,Colombia,Tradicional,Lavado,15.60,15.705128,13.15,Filtrado,180°-190°,9.36
525,01-100124,Jerico,Colombia,Tradicional,Lavado,15.00,13.333333,13.00,Filtrado,180°-190°,9.32
526,01-020424,Jerico,Colombia,Tradicional,Lavado,14.50,16.551724,12.10,Filtrado,180°-190°,9.36
527,01-200624,Jerico,Colombia,Tradicional,lavado,10.70,13.551402,9.25,Filtrado,180°-190°,9.28


In [93]:
df =  pd.merge(df_calidad, df_tostion, left_on='LOTE', right_on='Lote', how='inner')

tostion_typos = {
    'Beneficio': {
        'lavado': 'Lavado',
        'Lavado ': 'Lavado',
        'lavado ': 'Lavado',
    },
    'Perfil': {
        'Expressso': 'Espresso',
        'Expresso': 'Espresso',
        'Espresso': 'Espresso',
        'Espressso' : 'Espresso',
        'Espresso ': 'Espresso',
        'Filtrados': 'Filtrado',
        'Filtrado ': 'Filtrado',
        'Filtrados ' : 'Filtrado',
    },
    'Origen': {
        'Herrra ': 'Herrera',
        'Herrerra': 'Herrera',
        'Herrera ': 'Herrera',
        'Jerico ': 'Jerico',
        'Jericó ': 'Jerico'
    },
    'Proceso': {
        'Tradicional': 'Tradicional',
        'Tradicional ': 'Tradicional',
        'tradicional ' : 'Tradicional'
    },
    'Variedad': { 'Caturra ': 'Caturra' }
}
for columna, cambios in tostion_typos.items():
    df[columna] = df[columna].replace(cambios)

df[['Temp_inicio', 'Temp_final']] = df['Temp. De inicio y final'].str.extract(r'(\d+)[°º]*/(\d+)[°º]*').astype(float)
df['Tiempo de tueste'] = pd.to_datetime(
    df['Tiempo de tueste'],
    format='%H:%M:%S',    # ajusta si tu formato es distinto
    errors='coerce'       # convierte a NaT las cadenas mal formateadas
)

# Extraemos las horas, minutos y segundos
df['Tiempo de tueste'] = (
    df['Tiempo de tueste'].dt.hour * 60 +
    df['Tiempo de tueste'].dt.minute +
    df['Tiempo de tueste'].dt.second / 60
)
df

,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,PUNTAJE,acidez,as,ba,caramelo,...,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Temp. De inicio y final,Tiempo de tueste,Temp_inicio,Temp_final
0,01-190722,Madre Laura,765.0,10.9,14,84,0,0,0,0,...,Tradicional,Lavado,9.00,15.000000,7.65,Filtrado,175°/191°,481.0,175.0,191.0
1,01-190722,Madre Laura,765.0,10.9,14,84,0,0,0,0,...,Tradicional,Lavado,3.00,16.666667,2.50,Espresso,175°/195°,522.0,175.0,195.0
2,01-190722,Madre Laura,765.0,10.9,14,84,0,0,0,0,...,Tradicional,Lavado,9.00,16.444444,7.52,Filtrado,175°/190°,478.0,175.0,190.0
3,01-190722,Madre Laura,765.0,10.9,14,84,0,0,0,0,...,Tradicional,Lavado,11.70,16.324786,9.79,Filtrado,175°/191°,482.0,175.0,191.0
4,01-190722,Madre Laura,765.0,10.9,14,84,0,0,0,0,...,Tradicional,Lavado,29.00,15.344828,24.55,Filtrado,175°/192°,482.0,175.0,192.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,01-271023,Madre Laura,20.0,10.5,14,84,0,0,0,0,...,Tradicional,Lavado,15.60,15.705128,13.15,Filtrado,180°-190°,NaN,NaN,NaN
465,01-100124,Madre Laura,20.0,10.4,14,84,0,0,0,0,...,Tradicional,Lavado,15.00,13.333333,13.00,Filtrado,180°-190°,NaN,NaN,NaN
466,01-020424,Madre Laura,20.0,10.5,14,84,0,0,0,0,...,Tradicional,Lavado,14.50,16.551724,12.10,Filtrado,180°-190°,NaN,NaN,NaN
467,01-200624,Madre Laura,14.0,10.8,14,84,0,0,0,0,...,Tradicional,Lavado,10.70,13.551402,9.25,Filtrado,180°-190°,NaN,NaN,NaN
